# asf_search - Basic Overview
`asf_search` provides several helpful classes and functions to simplify working with the ASF catalog. This notebook briefly describes how to get started with `asf_search`.
***
## Before You Start
The steps outlined in this notebook assume `asf_search` is available on your system. `asf_search` is available through PyPi, Conda, and Github. For this demonstration, we have already installed `asf_search` within a virtual environment through PyPi via the command:
  
  
```pip install asf_search```
  
`asf_search` requires Python 3.6 or higher.

***
## Usage
Once installed, simply import `asf_search` as you would any other Python module:

In [1]:
import asf_search as asf

`asf_search` version numbers are based on [Semantic Versioning 2.0.0](https://semver.org/spec/v2.0.0.html):

In [24]:
asf.__version__

'3.0.6'

***

## Performing a Basic Search
[View this search in Vertex](https://search.asf.alaska.edu/#/?resultsLoaded=true&dataset=SENTINEL-1)

For this basic example, we will specify two search parameters: a platform, and how many results we want to retrieve:
- Sentinel-1
- 5 results max
    - (results are retrieved newest-first, so this will be the 5 newest products).

In [25]:
results = asf.search(platform=asf.PLATFORM.SENTINEL1, maxResults=5)

Alternatively, it may be useful to handle your search arguments as a dictionary:

In [26]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 5
}
results = asf.search(**opts)

Note the use of `asf_search`-provided constants, as many editors support use of these through autocompletion. Categories of constants include:
- `BEAMMODE`
- `FLIGHT_DIRECTION`
- `INSTRUMENT`
- `PLATFORM`
- `POLARIZATION`
- `PRODUCT_TYPE`
  


***
## Working With Results
### `ASFSearchResults`
Search results are returned as an `ASFSearchResults` object, a subclass of `UserList`, containing a list of `ASFProduct` objects, each of these classes providing some additional functionality to aid in working with the results and individual products:

In [27]:
results

ASFSearchResults([<asf_search.ASFProduct.ASFProduct at 0x109c362b0>,
                  <asf_search.ASFProduct.ASFProduct at 0x10a2c7160>])

### `ASFProduct`
`ASFProduct` provides a number of metadata fields, such as:
- Geographic coordinates
    - Latitude/Longitude
    - Shape type
- Scene and product metadata
    - Path, frame
    - Platform, beam, polarization
    - File name, size, URL
    - and many others
  
Geographic coordinates are stored in the `geometry` attribute:

In [28]:
results[0].geometry

{'coordinates': [[[-15.3625, 23.7706],
   [-19.2916, 23.3071],
   [-19.8695, 25.882],
   [-15.8563, 26.3442],
   [-15.3625, 23.7706]]],
 'type': 'Polygon'}

Other metadata is available through the `properties` attribute:

In [29]:
results[0].properties

{'beamModeType': 'EW',
 'browse': [],
 'bytes': '654310081',
 'faradayRotation': None,
 'fileID': 'S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF-RAW',
 'fileName': 'S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF.zip',
 'flightDirection': 'ASCENDING',
 'frameNumber': '76',
 'granuleType': 'SENTINEL_1A_FRAME',
 'groupID': 'S1A_EWDV_0076_0084_041957_060',
 'insarStackId': None,
 'md5sum': '688bd1933bc371b748d265338d996384',
 'offNadirAngle': None,
 'orbit': '41957',
 'pathNumber': '60',
 'platform': 'Sentinel-1A',
 'pointingAngle': None,
 'polarization': 'VV+VH',
 'processingDate': '2022-02-17T19:12:49.000000',
 'processingLevel': 'RAW',
 'sceneName': 'S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF',
 'sensor': 'C-SAR',
 'startTime': '2022-02-17T19:12:49.000000',
 'stopTime': '2022-02-17T19:13:32.000000',
 'url': 'https://datapool.asf.alaska.edu/RAW/SA/S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF.zip'}

### Output
The layout of the above data structure mirrors the geojson output format provided by ASF's SearchAPI, for a smooth transition to `asf_search`.
  
In fact, the `ASFSearchResults.__str__()` method serializes results to geojson _identical_ to that of ASF's SearchAPI, allowing for a drop-in replacement for users of the SearchAPI, simply by explicitly or implicitly casting search results to a string:

In [30]:
print(results)

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              -15.3625,
              23.7706
            ],
            [
              -19.2916,
              23.3071
            ],
            [
              -19.8695,
              25.882
            ],
            [
              -15.8563,
              26.3442
            ],
            [
              -15.3625,
              23.7706
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "EW",
        "browse": [],
        "bytes": "654310081",
        "faradayRotation": null,
        "fileID": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF-RAW",
        "fileName": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF.zip",
        "flightDirection": "ASCENDING",
        "frameNumber": "76",
        "granuleType": "SENTINEL_1A_FRAME",
        "groupID": "S1A_EWDV_0076_0084_041957_060"

Additionally, individual `ASFProduct` objects provides geojson-based serialization, in the form of a geojson `Feature` snippet:

In [31]:
print(results[0])

{
  "geometry": {
    "coordinates": [
      [
        [
          -15.3625,
          23.7706
        ],
        [
          -19.2916,
          23.3071
        ],
        [
          -19.8695,
          25.882
        ],
        [
          -15.8563,
          26.3442
        ],
        [
          -15.3625,
          23.7706
        ]
      ]
    ],
    "type": "Polygon"
  },
  "properties": {
    "beamModeType": "EW",
    "browse": [],
    "bytes": "654310081",
    "faradayRotation": null,
    "fileID": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF-RAW",
    "fileName": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF.zip",
    "flightDirection": "ASCENDING",
    "frameNumber": "76",
    "granuleType": "SENTINEL_1A_FRAME",
    "groupID": "S1A_EWDV_0076_0084_041957_060",
    "insarStackId": null,
    "md5sum": "688bd1933bc371b748d265338d996384",
    "offNadirAngle": null,
    "orbit": "41957",
    "pathNumber": "60",
    "platform": "Sentin

## Summary
A complete example, showing how simple use of `asf_search` can be:

In [32]:
import asf_search as asf

print(asf.search(platform='S1', maxResults=5))

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              -15.3625,
              23.7706
            ],
            [
              -19.2916,
              23.3071
            ],
            [
              -19.8695,
              25.882
            ],
            [
              -15.8563,
              26.3442
            ],
            [
              -15.3625,
              23.7706
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "EW",
        "browse": [],
        "bytes": "654310081",
        "faradayRotation": null,
        "fileID": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF-RAW",
        "fileName": "S1A_EW_RAW__0SDV_20220217T191249_20220217T191332_041957_04FF17_5DAF.zip",
        "flightDirection": "ASCENDING",
        "frameNumber": "76",
        "granuleType": "SENTINEL_1A_FRAME",
        "groupID": "S1A_EWDV_0076_0084_041957_060"

Next: [Geographic Searches](./2-Geographic_Search.ipynb)